In [1]:
import sys
import argparse

import torch
import torch.nn as nn
from torchtext.legacy import data

In [2]:
class RNNClassifier(nn.Module):
    def __init__(self,
                 input_size,
                 word_vec_size,
                 hidden_size,
                 n_classes,
                 n_layers = 4,
                 dropout_p = .3
                ):
        self.vocab_size = input_size # vocabulary_size
        self.word_vec_size = word_vec_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        super().__init__()
        
        self.emb = nn.Embedding(input_size, word_vec_size)
        self.rnn = nn.LSTM(input_size = word_vec_size,
                           hidden_size = hidden_size,
                           num_layers = n_layers,
                           dropout = dropout_p,
                           batch_first = True,
                           bidirectional = True
                          )
        self.generator = nn.Linear(hidden_size * 2, n_classes)
        # We use LogSoftmax + NLLLoss instead of Softmax + CrossEntropy
        self.activation = nn.LogSoftmax(dim = -1)
        
    def forward(self, x):
        # /x/ = (batch_size, length)
        x = self.emb(x)
        # /x/ = (batch_size, length, word_vec_dim)
        x, _ = self.rnn(x)
        # /x/ = (batch_size, length, hidden_size * 2)
        y = self.activation(self.generator(x[:, -1]))
        # /y/ = (batch_size, n_classes)
            
        return y
    
class Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.train_fn = "./data/processed_data.txt"
        
        self.gpu_id = -1
        self.verbose = 2
        
        self.min_vocab_freq = 5
        self.max_vocab_size = 999999
        
        self.batch_size = 256
        self.n_epochs = 10
        
        self.word_vec_size = 256
        self.dropout = .3
        
        self.max_length = 256
        
        self.rnn = True
        self.hidden_size = 512
        self.n_layers = 4
        
        self.cnn = False
        self.use_batch_norm = True
        self.window_sizes = [3, 4, 5]
        self.n_filters = [100, 100, 100]

In [3]:
def read_text(input_string, max_length=256):
    '''
    Read text from standard input for inference.
    '''
    lines = []

    for line in input_string: # sys.stdin:
        if line.strip() != '':
            lines += [line.strip().split(' ')[:max_length]]

    return lines

def define_field():
    '''
    To avoid use DataLoader class, just declare dummy fields. 
    With those fields, we can retore mapping table between words and indice.
    '''
    return (
        data.Field(
            use_vocab=True,
            batch_first=True,
            include_lengths=False,
        ),
        data.Field(
            sequential=False,
            use_vocab=True,
            unk_token=None,
        )
    )

In [4]:
class test_Config():
    def __init__(self):
        self.model_fn = "./models/model.pth"
        self.gpu_id = -1
        self.batch_size = 256
        self.top_k = 1
        self.max_length = 256
        
        self.drop_rnn = False
        self.drop_cnn = True

test_config = test_Config()

In [5]:
def Load_Model(input_string):
    saved_data = torch.load(
        test_config.model_fn,
        map_location='cpu' if test_config.gpu_id < 0 else 'cuda:%d' % test_config.gpu_id
    )
    
    train_config = saved_data['config']
    rnn_best = saved_data['rnn']
    cnn_best = saved_data['cnn']
    vocab = saved_data['vocab']
    classes = saved_data['classes']

    vocab_size = len(vocab)
    n_classes = len(classes)

    text_field, label_field = define_field()
    text_field.vocab = vocab
    label_field.vocab = classes

    lines = read_text(input_string, max_length=test_config.max_length)

    with torch.no_grad():
        ensemble = []
        if rnn_best is not None and not test_config.drop_rnn:
            # Declare model and load pre-trained weights.
            model = RNNClassifier(
                input_size=vocab_size,
                word_vec_size=train_config.word_vec_size,
                hidden_size=train_config.hidden_size,
                n_classes=n_classes,
                n_layers=train_config.n_layers,
                dropout_p=train_config.dropout,
            )
            model.load_state_dict(rnn_best)
            ensemble += [model]
        if cnn_best is not None and not test_config.drop_cnn:
            # Declare model and load pre-trained weights.
            model = CNNClassifier(
                input_size=vocab_size,
                word_vec_size=train_config.word_vec_size,
                n_classes=n_classes,
                use_batch_norm=train_config.use_batch_norm,
                dropout_p=train_config.dropout,
                window_sizes=train_config.window_sizes,
                n_filters=train_config.n_filters,
            )
            model.load_state_dict(cnn_best)
            ensemble += [model]

        y_hats = []
        # Get prediction with iteration on ensemble.
        for model in ensemble:
            if test_config.gpu_id >= 0:
                model.cuda(test_config.gpu_id)
            # Don't forget turn-on evaluation mode.
            model.eval()

            y_hat = []
            for idx in range(0, len(lines), test_config.batch_size):                
                # Converts string to list of index.
                x = text_field.numericalize(
                    text_field.pad(lines[idx:idx + test_config.batch_size]),
                    device='cuda:%d' % test_config.gpu_id if test_config.gpu_id >= 0 else 'cpu',
                )

                y_hat += [model(x).cpu()]
            # Concatenate the mini-batch wise result
            y_hat = torch.cat(y_hat, dim=0)
            # |y_hat| = (len(lines), n_classes)

            y_hats += [y_hat]

            model.cpu()
        # Merge to one tensor for ensemble result and make probability from log-prob.
        y_hats = torch.stack(y_hats).exp()
        # |y_hats| = (len(ensemble), len(lines), n_classes)
        y_hats = y_hats.sum(dim=0) / len(ensemble) # Get average
        # |y_hats| = (len(lines), n_classes)

        probs, indice = y_hats.topk(test_config.top_k)

        for i in range(len(lines)):
            sys.stdout.write('[%s, 점수:%s] %s\n' % (
                ' '.join(['긍정' if int(classes.itos[indice[i][j]]) > 3 else '부정' for j in range(test_config.top_k)]), 
                ' '.join([classes.itos[indice[i][j]] for j in range(test_config.top_k)]), 
                ' '.join(lines[i])
            ))

In [6]:
# input_string = [
#     "이건 쓰레기야",
#     "최고에요! ㅎㅎ",
#     "구매하길 잘 했네요",
#     "실망이에요.. 금방 망가지네요",
#     "다시는 구매 안합니다.",
#     "또 주문할게요.",
#     "포장도 제대로 안되어있고 정말 엉망이네요",
#     "내돈 돌려줘",
#     "사이즈가 안맞아요",
#     "따듯하고 좋아요 세탁하면 어떨지 아직은 모르겠네요 세탁후 줄어듬만 없으면 100점이에요",
#     "이정도면 무난히 만족합니다. 다른분들은 모르겠지만 딱 제가 찾던 폴라티네요.",
#     "남들이 욕하길래 정말 궁금해서 사봤어요 우와 세상에ㅎㅎ 잠옷이 생겼네요 요즘 좀 우울했는에 웃음이 나오네요ㅋㅋ 옷감은 무지 얇고 목은 왜이리 긴지 난 목이기니까 하면서 샀는데 어느나라에 목이 긴 부족에게 추천하고 싶네요 돈많고 좀 우울한 분에게 기분전환용으로 추천합니다 덕분에 웃었어요",
#     "다신 안삼",
#     "인연끊고 싶은 사람에게 추천해라",
#     "이런 입지도 못하는 옷을 팔다니 싸더라도 입을 수 있게는 만들어야지 기장이 짧아서 배꼽이 보일지경ㅜㅜ",
#     "한마디로 잘라 말하면 사지 마세요."
# ]

# input_string = [
#     "너무셔요 너무셔서애기가안먹네요 다른제품은잘먹었는데.. 제구매는없을듯요",
#     "오배송 5단계주문했는데 4단계왔습니다 안보고 모르고 한팩 뜯어썻네요 그거 제외하고 다시 교환해주세요",
#     "너무느려요 배송너무느려서 주문못하겠네요",
#     "정말 박스 너덜너덜하고 기저귀 터져서왔네요 제가 어지간해선 리뷰 귀찮아서 안남기는데 제개 보낸 기저귀는 박스도 재포장해서 너덜너덜해져서 거의 분리된상태로 온데다가 안에 뜯어보니 박스가 그상태로 굴러와서인지 원래 뜯어진건지, 시킨것중 기저귀한팩 바닥이 반쯤 뜯어져서 왔네요. 박스도 첨부터 너덜너덜해서 급히 그부분 보수해서 테이핑해서 온데다가, 안에 내용물까지 저모냥인데 그것도 다른게아니라 아기 소중한데 닿을 기저귀인데 정말 해도해도 너무하시네요. 동영상 가지고있구요, 저도 놀고먹는사람아니라서 바빠 반품하기도 싫습니다. 다음부터는 다른건 몰라도 아기용품은 양심껏 파세요 ㅡㅡ",
#     "배송진짜느림 배송진짜느려요 급한분들은 주문하시면 안될거같아요 피드백도 느려서 재고도없이 판매하면 어쩌라는건지;;",
#     "최악 포장도 엉망 cs도엉망 배송최악"
# ]

# input_string = [
#     "이거 정말 좋네요",
#     "배송도 빠르고 제품상태가 좋아요"
# ]

input_string = [
    "민경환 바보"
]

Load_Model(input_string)

[긍정, 점수:5] 민경환 바보
